In [4]:
!optimum-cli export onnx --model khanhld/wav2vec2-base-vietnamese-160h w2v2_onnx/

/home/spring/miniconda3/lib/python3.12/site-packages/torch/cuda/__init__.py:129: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'
/home/spring/miniconda3/lib/python3.12/site-packages/transformers/models/wav2vec2/modeling_wav2vec2.py:872: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  if attn_output.size() != (bsz, self.num_heads, tgt_len, self.head_dim):
		-[x] values not close enough, max diff: 8.33272933959961e-05 (atol: 1e-05)
The ONNX 

In [1]:
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
from optimum.onnxruntime import ORTModelForCTC, ORTModelForSpeechSeq2Seq
from jiwer import wer 
model = ORTModelForCTC.from_pretrained("w2v2_onnx/")
print(type(model))
processor = Wav2Vec2Processor.from_pretrained("khanhld/wav2vec2-base-vietnamese-160h")


/home/spring/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


<class 'optimum.onnxruntime.modeling_ort.ORTModelForCTC'>


In [2]:

#test model
import torch
import librosa
import numpy as np

def load_audio(file_path):
    speech_array, sampling_rate = librosa.load(file_path, sr=16000)
    inputs = processor(speech_array, sampling_rate=sampling_rate, return_tensors="pt", padding=True).input_values
    return inputs

def transcribe(file_path):
    input_values = load_audio(file_path)
    print(input_values.shape)
    logits = model(input_values).logits
    pred_ids = torch.argmax(logits, dim=-1)
    pred_transcript = processor.batch_decode(pred_ids)[0]
    
    return pred_transcript


preds = transcribe("wav/2.wav")
print(f"predicted:{preds}")
with open('txt/2.txt', 'r') as file:
    true = file.read()
    print(f"ground truth:{true}")
print(f"WER:{wer(preds, true)}")



torch.Size([1, 102080])
predicted:tướng ca pu chia còn yêu cầu thủ tướng sin ga po phải điều chỉnh phát biểu không đúng sự thật chút nào này theo lợi ông
ground truth:tướng cam pu chia còn yêu cầu thủ tướng sinh ga po phải điều chỉnh phát biểu không đúng sự thật chút nào này theo lời ông
WER:0.1111111111111111


## Evaluate performance:

In [25]:
#evaluate processing time
import time
times = []
for i in range(10):
    start = time.time()
    preds = transcribe(f"wav/{i}.wav")
    print(f"processing time:{time.time()-start}")
    print(f"transcribe: {preds}")
    end = time.time()
    times.append(end-start)


torch.Size([1, 139200])
processing time:0.7745380401611328
transcribe: hôm they hôm phu có nghĩa là chưa có cái giấy hôm thú nếu mà vậy thì cái cuộc bở lãnh phải làm coi như là cái người mà được ba lãnh họ phả ở nước ngoài học kế họ không
torch.Size([1, 50080])
processing time:0.20462536811828613
transcribe: theo bà đây là một sự thật đã được thừa nhận rộng rãi
torch.Size([1, 102080])
processing time:0.4502291679382324
transcribe: tướng ca pu chia còn yêu cầu thủ tướng sin ga po phải điều chỉnh phát biểu không đúng sự thật chút nào này theo lợi ông
torch.Size([1, 180320])
processing time:0.7073523998260498
transcribe: nó làm người dân mặc dù nhìn thủ rất bình thường nhớg mà rất rất là rộng ràng à thừ  từ từ tính tác đường phố và trên mỗi con mỗi con ngườn
torch.Size([1, 98240])
processing time:0.36347389221191406
transcribe: bây giờ mình nói từ khe xanh hai năm tới thể xanh vĩnh viễn ở mình đang ờ nói vì thẻ xanh nhưng mà có quá nhiều câu hỏi
torch.Size([1, 151360])
processing time:0.5

## Optimize the model 

In [5]:
from optimum.onnxruntime import ORTOptimizer, ORTModelForCTC, AutoOptimizationConfig
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC

processor = Wav2Vec2Processor.from_pretrained("khanhld/wav2vec2-base-vietnamese-160h")
model = ORTModelForCTC.from_pretrained("w2v2_onnx/")

optimizer = ORTOptimizer.from_pretrained(model)
save_dir_1 = 'w2v2_optim1'
save_dir_2 = 'w2v2_optim2'
save_dir_3 = 'w2v2_optim3'

optimization_config_1 = AutoOptimizationConfig.O1()
optimizer.optimize(save_dir=save_dir_1, optimization_config=optimization_config_1)

optimization_config_2 = AutoOptimizationConfig.O2()
optimizer.optimize(save_dir=save_dir_2, optimization_config=optimization_config_2)

optimization_config_3 = AutoOptimizationConfig.O3()
optimizer.optimize(save_dir=save_dir_3, optimization_config=optimization_config_3)




NotImplementedError: Tried to use ORTOptimizer for the model type wav2vec2, but it is not available yet. Please open an issue or submit a PR at https://github.com/huggingface/optimum.

## Quantize model

## Give TVM a shot cuz why not? (NVM)

In [21]:
import onnx
import numpy as np
import tvm.relay as relay
import tvm
from tvm.contrib import graph_executor
import librosa
#load w2v2_onnx model
model_path = "w2v2_onnx/model.onnx"
onnx_model = onnx.load(model_path)



In [23]:
target = "llvm"
input_name = 'input'
input_data = load_audio("wav/2.wav")
shape_dict = {input_name: [0,0]}
mod, params = relay.frontend.from_onnx(onnx_model, shape_dict)

with tvm.transform.PassContext(opt_level=3):
    lib = relay.build(mod, target, params=params)

dev = tvm.device(str(target), 0)
module = graph_executor.GraphModule(lib["default"](dev))

/home/spring/tvm/python/tvm/relay/frontend/onnx.py:7234: UserWarning: 'ORTModelForCTC' object has no attribute 'SerializeToString'
  warnings.warn(str(e))


AttributeError: 'ORTModelForCTC' object has no attribute 'graph'

In [16]:
import onnx

# Load your ONNX model
onnx_model = onnx.load(model_path)

# Print the input names
for input in onnx_model.graph.input:
    print(f"Input name: {input.name}")
    print(f"Input shape: {[dim.dim_value for dim in input.type.tensor_type.shape.dim]}")

Input name: input_values
Input shape: [0, 0]
